# About 
This notebook analyzes Boer-Mulders from CLAS data.  The following options can be configured.
- Include HERMES data or not 
- Run single fit or nested sampling
- Fit Collins or use the SIA result

In [1]:
import copy 
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
import sys

from fitlab.mcsamp import MCSAMP
from fitlab.maxlike import ML 
from fitlab.resman import RESMAN
from tools.config import load_config, conf
from tools.tools import load, save, checkdir

%matplotlib inline

In [2]:
if 'FITPACK' not in os.environ:
    print('Please set environment variable FITPACK to the root directory for this repo.')
else:
    print('Your FITPACK environment is %s' % os.environ['FITPACK'])

Your FITPACK environment is /home/dmriser/repos/fitpack


### Options Setup
The following cell is used to specify options that will be used in the notebook execution.

In [3]:
USE_HERMES   = False 
FIT_COLLINS  = False
FIT_TYPE     = 'nested'
path2results = 'sidis_boermulders_debug'

### Library Instantiation and Setup
The following cells are used for setting up the datasets, parameters, and fitter.  The only changes necessary are to the fixed/free status of the parameters and their boundaries.

In [4]:
checkdir(path2results)
conf['datasets'] = {}

In [5]:
def setup_datasets(dataset_indices):
    conf['datasets']['sidis']={}
    conf['datasets']['sidis']['filters']={0:{'idx':None,'filter':None}}
    conf['datasets']['sidis']['filters'][0]['filter']='z<0.6 and Q2>1.69 and pT>0.2 and pT<0.9'

    conf['datasets']['sidis']['filters'][0]['idx'] = []
    for index in dataset_indices:
        conf['datasets']['sidis']['filters'][0]['idx'].append(index)

    conf['datasets']['sidis']['xlsx']={}
    for dataset_index in conf['datasets']['sidis']['filters'][0]['idx']:
        conf["datasets"]["sidis"]["xlsx"][dataset_index] = 'sidis/expdata/%d.xlsx' % dataset_index     

    conf['datasets']['sidis']['norm']={}
    for k in conf['datasets']['sidis']['xlsx']: 
        conf['datasets']['sidis']['norm'][k]={'value':1,'fixed':True,'min':0,'max':1} 

In [6]:
# These datasets are the CLAS data.
active_datasets = [7002, 7003]

if USE_HERMES:
    active_datasets.extend([5019, 5020, 5023, 5024]) 
    
print('Loading... ', active_datasets)

('Loading... ', [7002, 7003])


In [7]:
setup_datasets(active_datasets)

# Let's double-check the list of datasets matches what we want.
for requested, added in zip(active_datasets, conf['datasets']['sidis']['xlsx'].keys()):
    assert (requested == added)

In [8]:
conf['params'] = {}
conf['shape']  = 1

In [9]:
def configure_collins_fixed():
    conf['params']['collins'] = {}
    conf['params']['collins']['widths0 pi+ fav']     = {'value':0.07249,'fixed':True,'min':1e-5,'max':1}
    conf['params']['collins']['widths0 pi+ unfav']   = {'value':0.10606,'fixed':True,'min':1e-5,'max':2}
    conf['params']['collins']['pi+ u N 1']  = {'value': 1.05988,'fixed':True,'min':0,'max':20}
    conf['params']['collins']['pi+ u a 1']  = {'value':-1.56637,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ u b 1']  = {'value': 4.76009 ,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ d N 1']  = {'value':-6.64443,'fixed':True,'min':-20,'max':0}
    conf['params']['collins']['pi+ d a 1']  = {'value': 3.45852,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ d b 1']  = {'value': 3.11503 ,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ u c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

    conf['params']['collins']['pi+ u N 2']  = {'value': 8.64876,'fixed':True,'min':0,'max':20}
    conf['params']['collins']['pi+ u a 2']  = {'value': 4.31424,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ u b 2']  = {'value': 3.59918,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ d N 2']  = {'value': 0.0,'fixed':True,'min':-20,'max':0}
    conf['params']['collins']['pi+ d a 2']  = {'value': 0.0,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ d b 2']  = {'value': 0.0,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ u c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

def configure_collins_free():
    conf['params']['collins']={}
    conf['params']['collins']['widths0 pi+ fav']     = {'value':7.47958632885192820083e-02,'fixed':False,'min':0.05,'max':0.10}
    conf['params']['collins']['widths0 pi+ unfav']   = {'value':0.059763576917398841815e+00,'fixed':False,'min':0.05,'max':0.136784756168045196212}
    conf['params']['collins']['pi+ u N 1']  = {'value': 4.49239476315589936206e+00,'fixed':False,'min':0,'max':3}
    conf['params']['collins']['pi+ u a 1']  = {'value':-8.28098465048009213518e-01,'fixed':False,'min':-1.9,'max':0}
    conf['params']['collins']['pi+ u b 1']  = {'value': 6.60780815284983358282e+00,'fixed':False,'min':2,'max':6}
    conf['params']['collins']['pi+ d N 1']  = {'value':-4.14852904854067539020e+00,'fixed':False,'min':-12,'max':0.0}
    conf['params']['collins']['pi+ d a 1']  = {'value': 1.00000000000000000000e+00,'fixed':False,'min': 0,'max':7.0}
    conf['params']['collins']['pi+ d b 1']  = {'value': 2.37348461151638101541e+00,'fixed':False,'min':2.5,'max':4.0}

    conf['params']['collins']['pi+ u c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 1']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

    conf['params']['collins']['pi+ u N 2']  = {'value': 6.11150078306284516572e+00,'fixed':False,'min':0,'max':15}
    conf['params']['collins']['pi+ u a 2']  = {'value': 4.71508421683099232524e+00,'fixed':False,'min':1,'max':8.0}
    conf['params']['collins']['pi+ u b 2']  = {'value': 2.75387728005980081392e+00,'fixed':False,'min':0,'max':5}

    conf['params']['collins']['pi+ d N 2']  = {'value': 0.0,'fixed':True,'min':-20,'max':0}
    conf['params']['collins']['pi+ d a 2']  = {'value': 0.0,'fixed':True,'min':-1,'max':5}
    conf['params']['collins']['pi+ d b 2']  = {'value': 0.0,'fixed':True,'min':1e-5,'max':10}
    conf['params']['collins']['pi+ u c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d c 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ u d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}
    conf['params']['collins']['pi+ d d 2']  = {'value': 0.0,'fixed':True,'min':-10,'max':10}

In [10]:
def configure_boermulders():
    conf['params']['boermulders']={}
    conf['params']['boermulders']['widths0 valence'] = {'value':2.99994257869736280497e-01,'fixed':False,'min':1e-5,'max':1}
    conf['params']['boermulders']['widths0 sea']     = {'value':2.32794366294785171068e-01,'fixed':False,'min':1e-5,'max':1}

    conf['params']['boermulders']['u N']  = {'value':5.42650693708019993267e+00,'fixed':False,'min':-10,'max':10}
    conf['params']['boermulders']['u a']  = {'value':1.61391852452949091656e+00,'fixed':False,'min':-1,'max':5}
    conf['params']['boermulders']['u b']  = {'value':1.13975971617359173038e+00,'fixed':False,'min':1e-5,'max':10}
    conf['params']['boermulders']['d N']  = {'value':7.33483204762133667032e+00,'fixed':False,'min':-10,'max':10}
    conf['params']['boermulders']['d a']  = {'value':1.96985180239880541819e-01,'fixed':False,'min':-1,'max':5}
    conf['params']['boermulders']['d b']  = {'value':1.00000000000000008180e-01,'fixed':False,'min':1e-5,'max':10}
    conf['params']['boermulders']['s N']  = {'value':1.00000000000000000000e+00,'fixed':False,'min':-10,'max':10}
    conf['params']['boermulders']['s a']  = {'value':1.00000000000000008180e+00,'fixed':False,'min':-1,'max':5}
    conf['params']['boermulders']['s b']  = {'value':1.00000000000000008180e+00,'fixed':False,'min':1e-5,'max':10}

    conf['params']['boermulders']['u c']  = {'value':0.00000000000000000000e+00,'fixed':True,'min':-10,'max':10}
    conf['params']['boermulders']['u d']  = {'value':0.00000000000000000000e+00,'fixed':True,'min':-10,'max':10}
    conf['params']['boermulders']['d c']  = {'value':0.00000000000000000000e+00,'fixed':True,'min':-10,'max':10}
    conf['params']['boermulders']['d d']  = {'value':0.00000000000000000000e+00,'fixed':True,'min':-10,'max':10}
    conf['params']['boermulders']['s c']  = {'value':0.00000000000000000000e+00,'fixed':True,'min':-10,'max':10}
    conf['params']['boermulders']['s d']  = {'value':0.00000000000000000000e+00,'fixed':True,'min':-10,'max':10}

In [11]:
configure_boermulders()

if FIT_COLLINS:
    configure_collins_free() 
else:
    configure_collins_fixed() 

In [12]:
def summarize_parameter_setup():
    for function_name in conf['params'].keys():
        print(function_name)
        
        for parameter in conf['params'][function_name]:
            fixed_status = 'fixed' if conf['params'][function_name][parameter]['fixed'] else 'free'
            print('{0:<24} {1:<8}'.format(parameter, fixed_status))

In [13]:
summarize_parameter_setup()

boermulders
s N                      free    
d c                      fixed   
d b                      free    
d a                      free    
d N                      free    
s b                      free    
s c                      fixed   
s d                      fixed   
u N                      free    
d d                      fixed   
s a                      free    
u d                      fixed   
widths0 sea              free    
u b                      free    
u c                      fixed   
widths0 valence          free    
u a                      free    
collins
pi+ d N 2                fixed   
pi+ d N 1                fixed   
widths0 pi+ unfav        fixed   
widths0 pi+ fav          fixed   
pi+ d c 2                fixed   
pi+ u a 2                fixed   
pi+ u b 2                fixed   
pi+ u b 1                fixed   
pi+ u N 2                fixed   
pi+ u N 1                fixed   
pi+ u c 2                fixed   
pi+ u c 1                fix

In [14]:
conf['ncpus']  = 1
conf['resman'] = RESMAN()
mcsamp         = MCSAMP()
maxlike        = ML() 

loading sidis data sets 7003
multiprocess setup: ncpus=1 / observable


### Start Running 
This cell will start the execution of the fitting.

In [16]:
if FIT_TYPE is 'single':
    maxlike.run_minimize(); 

elif FIT_TYPE is 'nested':
    conf['size']   = 2
    conf['factor'] = 1 
    mcsamp.run(path2results, 
           factor=conf['factor'], 
           size=conf['size'])
else:
    print('Fit type not recognized %s' % FIT_TYPE)

getting initial active p: 11/11

Process Process-3:
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/dmriser/repos/fitpack/fitlab/mcsamp.py", line 83, in single_run
    results=nest.NEST().run()
  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 406, in run
    self.next(t2-t1)
  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 361, in next
    self.gen_sample()
  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 336, in gen_sample
    _p,_nll=self.gen_par_cov(nll,p)
  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 3

KeyboardInterrupt: 

  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 336, in gen_sample
    _p,_nll=self.gen_par_cov(nll,p)
  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 305, in gen_par_cov
    ellipse.gen_new_samples()
  File "/home/dmriser/repos/fitpack/fitlab/nest.py", line 160, in gen_new_samples
    X=np.array([z[i]/r[i]*np.random.rand()**(1.0/self.dim) for i in range(self.N)])
KeyboardInterrupt


### To Do 
- Add plotting package for Boer-Mulders
- Add plotting for CLAS data
- Add plotting for HERMES data